In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
%%time
df = pd.read_csv('./data/train.csv', nrows=1e5, parse_dates=['click_time'])

CPU times: user 136 ms, sys: 17.5 ms, total: 153 ms
Wall time: 161 ms


In [4]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


In [5]:
df.set_index('click_time', inplace=True)

In [24]:
df.index.to_series().diff().dt.seconds

click_time
2017-11-06 14:32:21      NaN
2017-11-06 14:33:34     73.0
2017-11-06 14:34:12     38.0
2017-11-06 14:34:52     40.0
2017-11-06 14:35:08     16.0
2017-11-06 14:36:26     78.0
2017-11-06 14:37:44     78.0
2017-11-06 14:37:59     15.0
2017-11-06 14:38:10     11.0
2017-11-06 14:38:23     13.0
2017-11-06 14:38:27      4.0
2017-11-06 14:38:51     24.0
2017-11-06 14:39:29     38.0
2017-11-06 14:40:16     47.0
2017-11-06 14:40:39     23.0
2017-11-06 14:40:51     12.0
2017-11-06 14:43:10    139.0
2017-11-06 14:43:14      4.0
2017-11-06 14:43:25     11.0
2017-11-06 14:43:51     26.0
2017-11-06 14:44:02     11.0
2017-11-06 14:44:15     13.0
2017-11-06 14:44:51     36.0
2017-11-06 14:44:51      0.0
2017-11-06 14:44:52      1.0
2017-11-06 14:47:49    177.0
2017-11-06 14:48:07     18.0
2017-11-06 14:48:55     48.0
2017-11-06 14:49:36     41.0
2017-11-06 14:49:38      2.0
                       ...  
2017-11-06 16:01:48      0.0
2017-11-06 16:01:48      0.0
2017-11-06 16:01:48      0.0
201

In [12]:
df.groupby(['ip', 'app']).count().shape[0]

56782

In [14]:
df.groupby(['ip']).count().shape[0]

15369

In [11]:
df.shape[0]

100000